In [2]:
"""
实现二维互相关运算
"""
from mxnet import autograd, nd
from mxnet.gluon import nn

def corr2d(x, k):
    """
    :param x: 输入
    :param k: 卷积核
    :return: 计算后的矩阵
    """
    h, w = k.shape  # 获取卷积核尺寸
    y = nd.zeros((x.shape[0] - h + 1, x.shape[1] - w + 1))  # 计算通过这次卷积后生成的矩阵的大小，并生成相应大小的0矩阵
    """
    一行一行的计算得出y的值
    """
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (x[i: i + h, j: j + w] * k).sum()
    return y



In [3]:
"""
调用上面的函数验证
"""
x = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
k = nd.array([[0, 1], [2, 3]])
corr2d(x, k)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

In [4]:
corr2d(x, k).dtype

numpy.float32

In [5]:
"""
实现二维卷积层
"""
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        """
        生成这次卷积操作的，卷积核的数值，以及需要的偏置值。
        """
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias = self.params.get('bias',shape=(1,))  # 只需要一行，系统会利用广播机制扩展成契合的矩阵
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()


In [6]:
"""
应用举例
图像中物体边缘检测
"""
x = nd.ones((6, 8))
x[:, 2:6] = 0
x


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [7]:
#  构造卷积核
k = nd.array([[1, -1]])

In [8]:
#  执行卷积操作
y = corr2d(x, k)

In [10]:
y  # 由此可见卷积操作会在一定程度上得到一张图片的形状


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [12]:
conv2d = nn.Conv2D(1, kernel_size=(1, 2))
conv2d.initialize()
x = x.reshape((1, 1, 6, 8))  # 数据格式（样本数目，通道数， 高， 宽）
y = y.reshape((1, 1, 6, 7))

In [13]:
for i in range(10):
    with autograd.record():
        y_hat = conv2d(x)
        l = (y_hat  - y) ** 2
    l.backward()
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print('batch %d , loss %.3f' % (i + 1, l.sum().asscalar()))


batch 2 , loss 4.949
batch 4 , loss 0.831
batch 6 , loss 0.140
batch 8 , loss 0.024
batch 10 , loss 0.004


[14:38:28] /work/mxnet/src/base.cc:79: cuDNN lib mismatch: linked-against version 8801 != compiled-against version 8500.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


In [14]:
conv2d.weight.data().reshape(1, 2)


[[ 0.9895    -0.9873705]]
<NDArray 1x2 @cpu(0)>

In [ ]:
"""
特征图和感受野

"""